### Import Libraries

In [6]:
import pandas as pd
import numpy as np
import clickhouse_connect
from dotenv import dotenv_values

### Connect to ClickHouse DB

In [7]:
env_vars = dotenv_values('/root/text2sql/Credentials/.env')
host = env_vars['host']
port = int(env_vars['port'])
username = env_vars['user']
password = env_vars['password']

client = clickhouse_connect.get_client(host=host, port=port, secure=True, username=username, password=password)

### Define User Input

In [99]:
user_input = """"Get the best ad name by clicks from Facebook, Google, and LinkedIn for 2022 in terms of lowest CPC"""

### Import LangChain, LLM, and Create Prompt Template

In [8]:
# OpenAI
from langchain.llms import OpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

# API KEY
api_key = env_vars['OPENAI_API_KEY']

# LLM Model (GPT 3.5-Turbo)
llm = OpenAI(model_name='gpt-3.5-turbo', openai_api_key=api_key, temperature=0.0)

/root/text2sql/text2sql_env/lib/python3.10/site-packages/langchain/llms/openai.py:189: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/root/text2sql/text2sql_env/lib/python3.10/site-packages/langchain/llms/openai.py:769: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


### Define Context and Additional Contextual Definitions

In [101]:
context_query = """
You are a very experienced data engineer whose job is to write correct SQL queries. Given below is a description of four tables delimited with <>. The descriptions contain information about how tables were created,
what are the columns, their types, definitions, references to other tables using Foreign Keys and finally, first three rows as an example. 

<
Table 1: Users (Information about users, their registration date and activity status)

CREATE TABLE "Users" (
    "UserId" INTEGER NOT NULL (unique identifier of an user),
    "RegDate" DATE NOT NULL (date of registration),
    "Status" NVARCHAR(220) (status of the user: active or passive),
    PRIMARY KEY ("UserId")
)

SELECT * FROM "Users" LIMIT 3;
UserId RegDate Status
120	2023-03-04	passive
345	2023-03-19	active
533	2021-07-24	passive
>

<
Table 2: UserActivity (Information about users visit to the website. It contains history of dates of visitis, channels of visit: direct visit or through clicking an advertisement of marketing campaign. \ 
If visit happened by clicking the ad then corresponding campaign Id is also provided.)

CREATE TABLE "UserActivity" (
    "VisitId" INTEGER NOT NULL (unique identifier of a user's visit to website),
    "UserId" INTEGER NOT NULL (Id of an user),
    "VisitDate" DATE (date of visit),
    "Click" BOOLEAN (if user visited website after clicking an advertisement of marketing campaign on some platform (Google, LinkedIn, Facebook, Bing) then 1, otherwise 0),
    "CampaignId" INTEGER (Id of marketing campaign. If user arrived at website directly without advertisement then CampaignId is 999),
    PRIMARY KEY ("VisitId"),
    FOREIGN KEY("UserId") REFERENCES "Users" ("UserId"),
    FOREIGN KEY("CampaignId") REFERENCES "CampaignActivity" ("CampaignId")
)

SELECT * FROM "UserActivity" LIMIT 3;
VisitId UserId VisitDate Click CampaignId
23	5259	2021-11-27	1	25
24	708	    2023-05-18	1	29
46	7601	2022-11-04	0	7
>

<
Table 3: CampaignActivity (Information about unique marketing campaigns with starting and ending dates, cost of campaign and the platform where the advertisements/campaigns are/were running (LinkedIn, Google, Facebook, Bing))

CREATE TABLE "CampaignActivity" (
    "CampaignId" INTEGER NOT NULL (unique id of marketing campaign),
    "Platform" TEXT NOT NULL (a platform/social media where the advertisement/campaign is/was running),
    "AdStartDate" DATE (start date of advertisement/campaign),
    "AdEndDate" DATE (end date of advertisement/campaign),
    "Cost" REAL (cost of given advertisement/campaign in USD),
    PRIMARY KEY ("CampaignId")
)

SELECT * FROM "CampaignActivity" LIMIT 3;
CampaignId Platform AdStartDate AdEndDate Cost
1	Google	 2022-06-22	 2022-06-27	154.74
2	Facebook 2023-02-14	 2023-03-12	894.79
3	Google	 2022-12-20	 2023-01-18	897.17
>

<
TABLE 4: Customers (Information about clients/customers of marketing agency. Customers are not users. Customers pay money to marketing agency for advertisements/campaigns.)

CREATE TABLE "Customers" (
    "CustomerId" INTEGER NOT NULL (unique identifier of client/customer),
    "Name" TEXT NOT NULL (full name of the customer),
    "Email" TEXT NOT NULL (email of the customer),
    "Status" TEXT NOT NULL (status of the customer: active or passive),
    "CreatedAt" DATAE  (date of account creation/registration),
    PRIMARY KEY ("CustomerId")
)

SELECT * FROM "Customers" LIMIT 3;
CustomerId Name Email Status CreatedAt
36868	Ella Lewis	ella.lewis@example.com	inactive	2022-06-17
49449	Ava Miller	ava.miller@example.com	active	    2021-12-18
50287	Michael Rodriguez	michael.rodriguez@gmail.com	inactive	2022-07-03
>

Carefully analyze tables above and write proper SQL query for the following instructions delimited by triple backticks ```{user_text}```

For the definition of specific terminology you can use following: {def_terminology}

Write query in ClickHouse SQL.

Do not hallucinate. Don't use columns that aren't available in table. Use joins to other tables to find appropriate columns.

result must be just a sql query and nothing else!
"""

In [102]:
def_terminology = """
CPC - Cost per Click (calculated as sum of total cost of advertisement/campaign divided by total number of clicks)
"""

### Check Context Before Generating Query

In [103]:
db_context = f"""
<
Table 1: Users (Information about users, their registration date and activity status)

CREATE TABLE "Users" (
    "UserId" INTEGER NOT NULL (unique identifier of an user),
    "RegDate" DATE NOT NULL (date of registration),
    "Status" NVARCHAR(220) (status of the user: active or passive),
    PRIMARY KEY ("UserId")
)

SELECT * FROM "Users" LIMIT 3;
UserId RegDate Status
120	2023-03-04	passive
345	2023-03-19	active
533	2021-07-24	passive
>

<
Table 2: UserActivity (Information about users visit to the website. It contains history of dates of visitis, channels of visit: direct visit or through clicking an advertisement of marketing campaign. \ 
If visit happened by clicking the ad then corresponding campaign Id is also provided.)

CREATE TABLE "UserActivity" (
    "VisitId" INTEGER NOT NULL (unique identifier of a user's visit to website),
    "UserId" INTEGER NOT NULL (Id of an user),
    "VisitDate" DATE (date of visit),
    "Click" BOOLEAN (if user visited website after clicking an advertisement of marketing campaign on some platform (Google, LinkedIn, Facebook, Bing) then 1, otherwise 0),
    "CampaignId" INTEGER (Id of marketing campaign. If user arrived at website directly without advertisement then CampaignId is 999),
    PRIMARY KEY ("VisitId"),
    FOREIGN KEY("UserId") REFERENCES "Users" ("UserId"),
    FOREIGN KEY("CampaignId") REFERENCES "CampaignActivity" ("CampaignId")
)

SELECT * FROM "UserActivity" LIMIT 3;
VisitId UserId VisitDate Click CampaignId
23	5259	2021-11-27	1	25
24	708	    2023-05-18	1	29
46	7601	2022-11-04	0	7
>

<
Table 3: CampaignActivity (Information about unique marketing campaigns with starting and ending dates, cost of campaign and the platform where the advertisements/campaigns are/were running (LinkedIn, Google, Facebook, Bing))

CREATE TABLE "CampaignActivity" (
    "CampaignId" INTEGER NOT NULL (unique id of marketing campaign),
    "Platform" TEXT NOT NULL (a platform/social media where the advertisement/campaign is/was running),
    "AdStartDate" DATE (start date of advertisement/campaign),
    "AdEndDate" DATE (end date of advertisement/campaign),
    "Cost" REAL (cost of given advertisement/campaign in USD),
    PRIMARY KEY ("CampaignId")
)

SELECT * FROM "CampaignActivity" LIMIT 3;
CampaignId Platform AdStartDate AdEndDate Cost
1	Google	 2022-06-22	 2022-06-27	154.74
2	Facebook 2023-02-14	 2023-03-12	894.79
3	Google	 2022-12-20	 2023-01-18	897.17
>

<
TABLE 4: Customers (Information about clients/customers of marketing agency. Customers are not users. Customers pay money to marketing agency for advertisements/campaigns.)

CREATE TABLE "Customers" (
    "CustomerId" INTEGER NOT NULL (unique identifier of client/customer),
    "Name" TEXT NOT NULL (full name of the customer),
    "Email" TEXT NOT NULL (email of the customer),
    "Status" TEXT NOT NULL (status of the customer: active or passive),
    "CreatedAt" DATAE  (date of account creation/registration),
    PRIMARY KEY ("CustomerId")
)

SELECT * FROM "Customers" LIMIT 3;
CustomerId Name Email Status CreatedAt
36868	Ella Lewis	ella.lewis@example.com	inactive	2022-06-17
49449	Ava Miller	ava.miller@example.com	active	    2021-12-18
50287	Michael Rodriguez	michael.rodriguez@gmail.com	inactive	2022-07-03
>

For the definition of specific terminology you can use following: {def_terminology}
"""

In [104]:
context_checker = """"
You're the best data engineer in the world. You are an expert in analytics and SQL. Given the database tables described in triple backticks ```{db_context}```, can I write SQL query using tables mentiond above to \
asnwer the following question: {user_input}. 
your answer must be Yes or No and nothing more or less!
"""

In [105]:
prompt_context_checker = PromptTemplate.from_template(context_checker)
chain_context = LLMChain(llm=llm, prompt = prompt_context_checker)
result_context_checker = chain_context.run(db_context=db_context, user_input=user_input)
print(result_context_checker)

Yes.


### Develop Chain for Correct Query Generation

In [70]:
prompt_query = PromptTemplate.from_template(context_query)

In [71]:
chain_query = LLMChain(llm=llm, prompt = prompt_query)

result_query_init = chain_query.run(user_text=user_input, def_terminology=def_terminology)

In [72]:
print(result_query_init)

SELECT 
    ca.Platform,
    ca.CampaignId,
    ca.AdStartDate,
    ca.AdEndDate,
    SUM(ca.Cost) / COUNT(ua.VisitId) AS CPC
FROM 
    CampaignActivity ca
JOIN 
    UserActivity ua ON ca.CampaignId = ua.CampaignId
JOIN 
    Users u ON ua.UserId = u.UserId
WHERE 
    ca.Platform IN ('Facebook', 'Google', 'LinkedIn') AND
    ca.AdStartDate >= '2022-01-01' AND ca.AdEndDate <= '2022-12-31'
GROUP BY 
    ca.Platform,
    ca.CampaignId,
    ca.AdStartDate,
    ca.AdEndDate
ORDER BY 
    CPC ASC
LIMIT 1;


### Check The Syntax for Generated Query

In [94]:
sanity_check = """
You're the best data engineer in the world. You are an expert in SQL. Given the SQL query in triple backticks ```{result_query_init}```. \
If the syntax of query is correct then return it without any changes, otherwise make apporpriate changes and return new query.
Your output in both cases must be query only!
"""

In [95]:
prompt_sanity = PromptTemplate.from_template(sanity_check)

chain_sanity = LLMChain(llm=llm, prompt = prompt_sanity)

result_query_final = chain_sanity.run(result_query_init = result_query_init)

In [96]:
print(result_query_final)

SELECT 
    ca.Platform,
    ca.CampaignId,
    ca.AdStartDate,
    ca.AdEndDate,
    SUM(ca.Cost) / COUNT(ua.VisitId) AS CPC
FROM 
    CampaignActivity ca
JOIN 
    UserActivity ua ON ca.CampaignId = ua.CampaignId
JOIN 
    Users u ON ua.UserId = u.UserId
WHERE 
    ca.Platform IN ('Facebook', 'Google', 'LinkedIn') AND
    ca.AdStartDate >= '2022-01-01' AND ca.AdEndDate <= '2022-12-31'
GROUP BY 
    ca.Platform,
    ca.CampaignId,
    ca.AdStartDate,
    ca.AdEndDate
ORDER BY 
    CPC ASC
LIMIT 1;


### Run The Final Query Against Database

In [97]:
result_from_db = client.command(result_query_final)
print(result_from_db)

['LinkedIn', '25', '2022-09-26', '2022-10-19', '324.010009765625']


### Evaluation

In [44]:
# Manually checked text and query pairs

evaluation_dict = dict({
  "How many customers does the agency have?": "SELECT COUNT(*) AS total_customers FROM `Customers`;",
  "What is the total cost of all marketing campaigns?": "SELECT SUM(Cost) AS total_cost FROM `CampaignActivity`;",
  "How many active users does the platform currently have?": "SELECT COUNT(*) AS active_users FROM `Users` WHERE Status = 'active';",
  "Which campaign generated the highest user activity?": "SELECT ca.CampaignId, COUNT(ua.VisitId) AS activity_count FROM `CampaignActivity` AS ca JOIN `UserActivity` AS ua ON ca.CampaignId = ua.CampaignId WHERE ua.VisitDate >= '2021-01-01' AND ua.VisitDate <= '2023-12-31' GROUP BY ca.CampaignId ORDER BY activity_count DESC LIMIT 1;",
  "What is the total cost of campaigns on each platform?": "SELECT Platform, SUM(Cost) AS platform_total_cost FROM `CampaignActivity` WHERE AdStartDate >= '2021-01-01' AND AdEndDate <= '2023-12-31' GROUP BY Platform;",
  "How many customers were acquired each month?": "SELECT toStartOfMonth(CreatedAt) AS month, COUNT(*) AS customer_count FROM `Customers` WHERE CreatedAt >= '2021-01-01' AND CreatedAt <= '2023-12-31' GROUP BY month ORDER BY month;",
  "What is the average number of visits per user?": "WITH user_visits AS (SELECT UserId, COUNT(*) AS visit_count FROM `UserActivity` WHERE VisitDate >= '2021-01-01' AND VisitDate <= '2023-12-31' GROUP BY UserId) SELECT AVG(visit_count) AS average_visits_per_user FROM user_visits;",
  "Which user has the highest number of visits?": "SELECT ua.UserId, COUNT(*) AS visit_count FROM `UserActivity` AS ua WHERE ua.VisitDate >= '2021-01-01' AND ua.VisitDate <= '2023-12-31' GROUP BY ua.UserId ORDER BY visit_count DESC LIMIT 1;",
  "How many users signed up in each month of the current year?": "SELECT toStartOfMonth(RegDate) AS month, COUNT(*) AS signups_per_month FROM `Users` WHERE RegDate >= '2021-01-01' AND RegDate <= '2023-12-31' GROUP BY month ORDER BY month;",
  "Which customers have not been active in the last 60 days?": "SELECT ac.CustomerId, ac.Name FROM `Customers` AS ac LEFT JOIN `UserActivity` AS ua ON ac.CustomerId = ua.UserId WHERE (ua.VisitDate < NOW() - INTERVAL 60 DAY OR ua.VisitDate IS NULL) AND ac.CreatedAt >= '2021-01-01' AND ac.CreatedAt <= '2023-12-31';",
}
)

In [45]:
# Create a DataFrame from the dictionary
df_evaluation = pd.DataFrame(list(evaluation_dict.items()), columns=['Text', 'Query'])

In [57]:
# df_evaluation.head()

In [58]:
# for query in df_evaluation['Query']:
#     print(query)

In [48]:
df_evaluation.to_csv('evaluation_dataset.csv')

In [50]:
correct_output = []

for input_query in df_evaluation['Query']:
    corr_output = client.command(input_query)
    correct_output.append(corr_output)


for t in correct_output:
    print(t)

100
16270.039817810059
26
['27', '25']
['Facebook', '2855.4999618530273\nBing', '6953.599899291992\nLinkedIn', '3446.699981689453\nGoogle', '3014.239974975586']
['2021-11-01', '2\n2021-12-01', '7\n2022-01-01', '4\n2022-02-01', '3\n2022-03-01', '3\n2022-04-01', '7\n2022-05-01', '9\n2022-06-01', '5\n2022-07-01', '7\n2022-08-01', '3\n2022-09-01', '4\n2022-10-01', '1\n2022-11-01', '4\n2022-12-01', '6\n2023-01-01', '11\n2023-02-01', '3\n2023-03-01', '6\n2023-04-01', '1\n2023-05-01', '6\n2023-06-01', '8']
20
['1839', '30']
['2021-01-01', '2\n2021-02-01', '2\n2021-03-01', '2\n2021-06-01', '1\n2021-07-01', '2\n2021-08-01', '3\n2021-09-01', '1\n2021-11-01', '1\n2021-12-01', '2\n2022-01-01', '3\n2022-02-01', '3\n2022-03-01', '1\n2022-04-01', '2\n2022-05-01', '1\n2022-06-01', '2\n2022-09-01', '4\n2022-10-01', '1\n2022-12-01', '3\n2023-01-01', '4\n2023-02-01', '1\n2023-03-01', '5\n2023-04-01', '2\n2023-05-01', '1\n2023-06-01', '1']
['36868', 'Ella Lewis\n49449', 'Ava Miller\n50287', 'Michael Rodri

In [56]:
# Evaluation dataframe: Version 2
df_evaluation_v2 = df_evaluation.copy()
df_evaluation_v2['CorrectOutput'] = correct_output
df_evaluation_v2.head(10)

,Text,Query,CorrectOutput
0,How many customers does the agency have?,SELECT COUNT(*) AS total_customers FROM `Custo...,100
1,What is the total cost of all marketing campai...,SELECT SUM(Cost) AS total_cost FROM `CampaignA...,16270.039817810059
2,How many active users does the platform curren...,SELECT COUNT(*) AS active_users FROM `Users` W...,26
3,Which campaign generated the highest user acti...,"SELECT ca.CampaignId, COUNT(ua.VisitId) AS act...","[27, 25]"
4,What is the total cost of campaigns on each pl...,"SELECT Platform, SUM(Cost) AS platform_total_c...","[Facebook, 2855.4999618530273\nBing, 6953.5998..."
5,How many customers were acquired each month?,"SELECT toStartOfMonth(CreatedAt) AS month, COU...","[2021-11-01, 2\n2021-12-01, 7\n2022-01-01, 4\n..."
6,What is the average number of visits per user?,"WITH user_visits AS (SELECT UserId, COUNT(*) A...",20
7,Which user has the highest number of visits?,"SELECT ua.UserId, COUNT(*) AS visit_count FROM...","[1839, 30]"
8,How many users signed up in each month of the ...,"SELECT toStartOfMonth(RegDate) AS month, COUNT...","[2021-01-01, 2\n2021-02-01, 2\n2021-03-01, 2\n..."
9,Which customers have not been active in the la...,"SELECT ac.CustomerId, ac.Name FROM `Customers`...","[36868, Ella Lewis\n49449, Ava Miller\n50287, ..."


In [62]:
# Predition Dataframe
df_prediction = pd.read_csv('df_prediction.csv')
df_prediction.head(10)

,Unnamed: 0,Text,Prediction
0,0,How many customers does the agency have?,100
1,1,What is the total cost of all marketing campai...,16270.039817810059
2,2,How many active users does the platform curren...,"Error ('Connection aborted.', RemoteDisconnect..."
3,3,Which campaign generated the highest user acti...,"['999', '531']"
4,4,What is the total cost of campaigns on each pl...,"['Facebook', '41821.369300842285', '572.895469..."
5,5,How many customers were acquired each month?,"['2021-11-01', '2\n2021-12-01', '7\n2022-01-01..."
6,6,What is the average number of visits per user?,20
7,7,Which user has the highest number of visits?,"['1839', '30']"
8,8,How many users signed up in each month of the ...,:HTTPDriver for https://ppio7dt65z.europe-west...
9,9,Which customers have not been active in the la...,"['Ella Lewis', 'ella.lewis@example.com\nAva Mi..."


In [63]:
# Define propmpt
evaluation_promp = """ 
I have manually checked output of SQL query that I wrote. I also have an output that was generated by LLM model using writing appropriate SQL query. I want you to take these two outputs delimited by <>, \
compare them to each other and if they are the same then return True other False.

input 1: <{manual_output}>
input 2: <{model_output}>

Remember, return only True or False and nothing else. No other text, comment, assessment, suggestion etc. Either True or False!
"""

In [65]:
# Define a prompt template
prompt_evaluation = PromptTemplate.from_template(evaluation_promp)
chain_evaluation = LLMChain(llm=llm, prompt = prompt_evaluation)

In [68]:
df_prediction.columns, df_evaluation_v2.columns

(Index(['Unnamed: 0', 'Text', 'Prediction'], dtype='object'),
 Index(['Text', 'Query', 'CorrectOutput'], dtype='object'))

In [72]:
for manual_output, model_output in zip(df_evaluation_v2['CorrectOutput'].to_list(), df_prediction['Prediction'].to_list()):
    print("manual_output:", manual_output)
    print("model_output:" , model_output)

manual_output: 100
manual_output: 16270.039817810059
manual_output: 26
manual_output: ['27', '25']
manual_output: ['Facebook', '2855.4999618530273\nBing', '6953.599899291992\nLinkedIn', '3446.699981689453\nGoogle', '3014.239974975586']
manual_output: ['2021-11-01', '2\n2021-12-01', '7\n2022-01-01', '4\n2022-02-01', '3\n2022-03-01', '3\n2022-04-01', '7\n2022-05-01', '9\n2022-06-01', '5\n2022-07-01', '7\n2022-08-01', '3\n2022-09-01', '4\n2022-10-01', '1\n2022-11-01', '4\n2022-12-01', '6\n2023-01-01', '11\n2023-02-01', '3\n2023-03-01', '6\n2023-04-01', '1\n2023-05-01', '6\n2023-06-01', '8']
manual_output: 20
manual_output: ['1839', '30']
manual_output: ['2021-01-01', '2\n2021-02-01', '2\n2021-03-01', '2\n2021-06-01', '1\n2021-07-01', '2\n2021-08-01', '3\n2021-09-01', '1\n2021-11-01', '1\n2021-12-01', '2\n2022-01-01', '3\n2022-02-01', '3\n2022-03-01', '1\n2022-04-01', '2\n2022-05-01', '1\n2022-06-01', '2\n2022-09-01', '4\n2022-10-01', '1\n2022-12-01', '3\n2023-01-01', '4\n2023-02-01', '1\n

In [73]:
res_evaluation = []
for manual_output, model_output in zip(df_evaluation_v2['CorrectOutput'].to_list(), df_prediction['Prediction'].to_list()):
    result_context_checker = chain_context.run(manual_output=manual_output, model_output=model_output)
    res_evaluation.append(result_context_checker)

In [74]:
res_evaluation

['True',
 'True',
 'False',
 'False',
 'False',
 'True',
 'True',
 'True',
 'False',
 'False']